# Yolo v3 en Tensorflow y Keras

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization,  LeakyReLU, ZeroPadding2D, UpSampling2D
from tensorflow.keras.layers import Add, Concatenate
from tensorflow.keras import models
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import plot_model

## 1. Darknet 53

In [2]:
def DarknetConv2D(x,*args, **kwargs):
  darknet_kwargs = {'kernel_regularizer': l2(5e-4)}
  darknet_kwargs['padding'] = 'valid' if kwargs.get('strides')==(2,2) else 'same'
  darknet_kwargs.update(kwargs)
  return Conv2D(*args, **darknet_kwargs)(x)

In [3]:
def DarknetConv2D_BN_LR(x,*args, **kwargs):
  no_bias_kwargs = {'use_bias':False}
  no_bias_kwargs.update(kwargs)
  idx = no_bias_kwargs.pop('idx')
  x = DarknetConv2D(x,*args, **no_bias_kwargs, name='conv_'+str(idx))
  x = BatchNormalization(name='bnorm_'+str(idx))(x)
  x = LeakyReLU(alpha=0.1,name='leaky_'+str(idx))(x)
  return x

In [4]:
def DarknetResidual(x, filters, blocks, idx):
  x = ZeroPadding2D(((1,0),(1,0)))(x)
  x = DarknetConv2D_BN_LR(x, filters, 3, strides=(2,2), idx=idx)
  for i in range(blocks):
    idx += 1
    y = DarknetConv2D_BN_LR(x,filters//2, 1, idx=idx)
    idx += 1
    y = DarknetConv2D_BN_LR(y,filters, 3, idx=idx)
    idx += 1
    x = Add()([x,y])
  return x, idx+1

In [5]:
def DarknetBody(x):
  x = DarknetConv2D_BN_LR(x,32, 3, idx=0) # Conv 0
  x, idx = DarknetResidual(x,64,1, idx=1) # Conv 1-4
  x, idx = DarknetResidual(x,128,2, idx=idx)
  x, idx = DarknetResidual(x,256,8, idx=idx)
  skip_36 = x
  x, idx = DarknetResidual(x,512,8, idx=idx)
  skip_61 = x
  x, idx = DarknetResidual(x,1024,4, idx=idx) # Linea 547 yolov3.cfg
  return x, idx, [skip_36, skip_61]

In [6]:
def LastLayers(x, filters, out_filters, idxs):
  x = DarknetConv2D_BN_LR(x,filters, 1, idx = idxs[0])
  x = DarknetConv2D_BN_LR(x,2*filters, 3, idx = idxs[1])
  x = DarknetConv2D_BN_LR(x,filters, 1, idx = idxs[2])
  x = DarknetConv2D_BN_LR(x,2*filters, 3, idx = idxs[3])
  x = DarknetConv2D_BN_LR(x,filters, 1, idx = idxs[4])

  y = DarknetConv2D_BN_LR(x,2*filters, 3, idx = idxs[5])
  y = DarknetConv2D(y,out_filters, 1, name='conv_'+str(idxs[6]))
  return x,y

In [7]:
def Yolov3Body(inputs, n_anchors, n_class):
  x, idx, skips = DarknetBody(inputs)
  # skips: [skip_36, skip_61]
  darknet = models.Model(inputs, x)
  idxs = list(range(idx,idx+7))
  x, yolo_82 = LastLayers(darknet.output, 512, n_anchors*(n_class+5), idxs)
  x = DarknetConv2D_BN_LR(x,256, 1, idx=84)
  x = UpSampling2D(2)(x)
  x = Concatenate()([x,skips[1]])

  idxs = list(range(87,94))
  x, yolo_94 = LastLayers(x, 256, n_anchors*(n_class+5), idxs)
  x = DarknetConv2D_BN_LR(x,128, 1, idx=96)
  x = UpSampling2D(2)(x)
  x = Concatenate()([x,skips[0]])

  idxs = list(range(99,106))
  x, yolo_106 = LastLayers(x, 128, n_anchors*(n_class+5), idxs)
  
  return models.Model(inputs, [yolo_82,yolo_94,yolo_106])

## 2. Yolov3

In [8]:
inputs = Input(shape=(416,416,3))
model = Yolov3Body(inputs, 3, 80)

In [9]:
len(model.layers)

252

In [ ]:
model.summary()

Model: "model_28"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_23 (InputLayer)          [(None, 416, 416, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv_0 (Conv2D)                (None, 416, 416, 32  864         ['input_23[0][0]']               
                                )                                                                 
                                                                                                  
 bnorm_0 (BatchNormalization)   (None, 416, 416, 32  128         ['conv_0[0][0]']                 
                                )                                                          

In [10]:
plot_model(model, show_shapes=True,to_file='./modelo.png')

Output hidden; open in https://colab.research.google.com to view.

## 3. Lectura de pesos

In [ ]:
! wget https://pjreddie.com/media/files/yolov3.weights

--2022-06-26 19:20:00--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  22.9MB/s    in 11s     

2022-06-26 19:20:12 (21.5 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [ ]:
import struct
import numpy as np

In [ ]:
class ReadWeights:
  def __init__(self, weights_file):
    with open(weights_file, 'rb') as wf:
      major, = struct.unpack('i',wf.read(4))
      minor, = struct.unpack('i',wf.read(4))
      revision, = struct.unpack('i',wf.read(4))
      if (major*10 + minor) >= 2 and major < 1000 and minor <1000:
        print(wf.read(8))
      else:
        print(wf.read(4))
      transpose = (major>1000) or (minor > 1000)
      binary = wf.read()
    self.weights = np.frombuffer(binary, dtype='float32')
    self.offset = 0
  
  def read_weights(self, size):
    weights = self.weights[self.offset:self.offset + size]
    self.offset = self.offset + size
    return weights

  def load_weights(self, model):
    for i in range(106):
      try:
        layer_conv = model.get_layer('conv_'+str(i))
        print("Asignando pesos a la capa",i)
        if i not in [81, 93, 105]:
          layer_bn = model.get_layer('bnorm_'+str(i))
          size = np.prod(layer_bn.get_weights()[0].shape)
          beta = self.read_weights(size)
          gamma = self.read_weights(size)
          mean = self.read_weights(size)
          var = self.read_weights(size)
          layer_bn.set_weights([gamma, beta, mean, var])
        if len(layer_conv.get_weights()) > 1:
          # Tiene bias
          bias_size = np.prod(layer_conv.get_weights()[1].shape)
          bias = self.read_weights(bias_size)
          kernel_size = np.prod(layer_conv.get_weights()[0].shape)
          kernel = self.read_weights(kernel_size)
          kernel = kernel.reshape(list(reversed(layer_conv.get_weights()[0].shape)))
          kernel = kernel.transpose([2,3,1,0])
          layer_conv.set_weights([kernel, bias])
        else:
          kernel_size = np.prod(layer_conv.get_weights()[0].shape)
          kernel = self.read_weights(kernel_size)
          kernel = kernel.reshape(list(reversed(layer_conv.get_weights()[0].shape)))
          kernel = kernel.transpose([2,3,1,0])
          layer_conv.set_weights([kernel])
      except:
        print('Ocurrio un problema')
  def reset(self):
    self.offset = 0

In [ ]:
rw = ReadWeights('/content/yolov3.weights')

b'\x00|\xe8\x01\x00\x00\x00\x00'


In [ ]:
rw.load_weights(model)

Asignando pesos a la capa 0
Asignando pesos a la capa 1
Asignando pesos a la capa 2
Asignando pesos a la capa 3
Ocurrio un problema
Asignando pesos a la capa 5
Asignando pesos a la capa 6
Asignando pesos a la capa 7
Ocurrio un problema
Asignando pesos a la capa 9
Asignando pesos a la capa 10
Ocurrio un problema
Asignando pesos a la capa 12
Asignando pesos a la capa 13
Asignando pesos a la capa 14
Ocurrio un problema
Asignando pesos a la capa 16
Asignando pesos a la capa 17
Ocurrio un problema
Asignando pesos a la capa 19
Asignando pesos a la capa 20
Ocurrio un problema
Asignando pesos a la capa 22
Asignando pesos a la capa 23
Ocurrio un problema
Asignando pesos a la capa 25
Asignando pesos a la capa 26
Ocurrio un problema
Asignando pesos a la capa 28
Asignando pesos a la capa 29
Ocurrio un problema
Asignando pesos a la capa 31
Asignando pesos a la capa 32
Ocurrio un problema
Asignando pesos a la capa 34
Asignando pesos a la capa 35
Ocurrio un problema
Asignando pesos a la capa 37
Asign

Guardamos el modelo

In [ ]:
model.save('yolov3.h5')

In [ ]:
! cp /content/yolov3.h5 /content/drive/MyDrive/clases/"Computer Vision Modulo 1"/"Clase 8"/yolov3.h5

Funciones para extraer los Bounding Box

In [ ]:
def YoloCorrectBoxes(box_xy, box_wh, input_shape, image_shape):
  box_yx = box_xy[..., ::-1]
  box_hw = box_wh[..., ::-1] 
  input_shape = tf.cast(input_shape, tf.dtype(box_yx))
  image_shape = tf.cast(input_shape, tf.dtype(box_yx))
  new_shape = tf.round(image_shape* tf.min(input_shape/image_shape))
  offset = (input_shape - new_shape)/input_shape/2.
  scale = input_shape/new_shape
  box_yx = (box_yx - offset)*scale
  box_hw *= scale

  box_min = box_yx - (box_hw/2.)
  box_max = box_yx + (box_hw/2.)
  # y_min, x_min, y_max, x_max 
  boxes = tf.concatenate([box_min[...,0:1], box_min[...,1:2], box_max[...,0:1], box_max[...,1:2]])
  boxes *= tf.concatenate([image_shape, image_shape])
  return boxes

In [ ]:
def YoloScoreBoxes(feats, anchors, n_class, input_shape, image_shape):
  box_xy, box_wh, box_confidence, box_class = YoloLast(feats, anchors, n_class, input_shape)
  boxes = YoloCorrectBoxes(box_xy, box_wh, input_shape, image_shape)
  boxes = tf.reshape(boxes, [-1,4])
  box_scores = box_confidence * box_class
  box_scores = tf.reshape(box_scores,[-1, n_class])
  return boxes, box_scores